In [1]:
import pandas as pd
data = pd.read_csv('water_potability.csv') #This works becuase the 
    #Jupyter Notebook is already in the folder where the data file lives.
#print(data)

In [ ]:
data.info()

In [ ]:
data.describe()

In [83]:
data.shape

(3276, 10)

In [ ]:
data.dropna().shape

In [ ]:
2011/3276

In [ ]:
#60 percent of the rows have no nulls.
#So just dropping everything seems like a bad idea.
#So create a pipeline to test imputation methods
    #[split into train and test]
    #impute for each column that needs it - ph, Sulfate, Trihalomethanes
    #normalize
    #run through regressions
        #logistic
        #SVM
        #KNN
        #RF

In [2]:
#Split the train data into train and val
from sklearn.model_selection import train_test_split

train_set, val_set = train_test_split(data, test_size=.4, random_state=42)
#print("train shape = ", train_set.shape)
#print("val shape = ", val_set.shape)

In [3]:
valid_set, test_set = train_test_split(val_set, test_size = .5, random_state = 42)
#print("train shape = ", train_set.shape)
#print("val shape = ", val_set.shape)
#print("valid shape = ", valid_set.shape)
#print("test shape = ", test_set.shape)

In [4]:
train_set_values = train_set.pop('Potability') #remove the column from the df and save it elsewhere
valid_set_values = valid_set.pop('Potability') #remove the column from the df and save it elsewhere
test_set_values = test_set.pop('Potability') #remove the column from the df and save it elsewhere

#print("train shape = ", train_set.shape)
#print("val shape = ", val_set.shape)
#print("valid shape = ", valid_set.shape)
#print("test shape = ", test_set.shape)

#print("train values shape = ", train_set_values.shape)
#print("valid values shape = ", valid_set_values.shape)
#print("test values shape = ", test_set_values.shape)

In [ ]:
1965+655+656

In [ ]:
#from sklearn.preprocessing import StandardScaler
#scale = StandardScaler()
#scaled = scale.fit_transform(train_set)
#print(scaled)
#So it will work to just throw the dataset at it. That's what I wanted to know.

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [19]:
#logistic pipe
from sklearn.linear_model import LogisticRegression
full_pipe_log = Pipeline([
    ('imputer', SimpleImputer()), #there will, ultimately, be three options here.
        #mean, median, and node
    ('scaler', StandardScaler()), #this is statistical normalization (z - mu)/s.d.
    ('regressor', LogisticRegression())
])

param_grid = {
        'imputer__strategy': ["most_frequent","mean", "median"],
        'regressor__C': [10**10,10**9,10**8],
        'regressor__penalty': ['l1', 'l2']
        }

method_search = GridSearchCV(
    full_pipe_log
    , param_grid
    , cv = 5
    , scoring = 'neg_mean_squared_error'
    , verbose = 4
    , n_jobs = 6)
method_search.fit(train_set, train_set_values)
print(method_search.best_params_)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done  79 out of  90 | elapsed:    0.3s remaining:    0.0s


{'imputer__strategy': 'mean', 'regressor__C': 10000000000, 'regressor__penalty': 'l2'}


[Parallel(n_jobs=6)]: Done  90 out of  90 | elapsed:    0.4s finished


In [20]:
import numpy as np
pred_values = method_search.predict(valid_set)
ct = np.array(pd.crosstab(pred_values, valid_set_values))
print(pred_values)
print(ct)
print(ct.trace()/ct.sum().sum())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
#.639 is what I had without messing with hyperparameters

In [79]:
#SVM pipe
from sklearn.svm import SVC
full_pipe_SV = Pipeline([
    ('imputer', SimpleImputer()), #there will, ultimately, be three options here.
        #mean, median, and node
    ('scaler', StandardScaler()), #this is statistical normalization (z - mu)/s.d.
    ('regressor', SVC()) #will need to specify kernel type (linear, polynomial, RBF)
])
#Also specify regularization term C for how much error is OK
#Also specify gamma for tightness of fit
param_grid = {
        'imputer__strategy': ["most_frequent","mean", "median"],
        'regressor__C': [10**1,10**0,10**-1]
        }

method_search = GridSearchCV(
    full_pipe_SV
    , param_grid
    , cv = 5
    , scoring = 'neg_mean_squared_error'
    , verbose = 4
    , n_jobs = 6)
method_search.fit(train_set, train_set_values)
print(method_search.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.6s


{'imputer__strategy': 'mean', 'regressor__C': 1}


[Parallel(n_jobs=6)]: Done  45 out of  45 | elapsed:    1.3s finished


In [80]:
import numpy as np
pred_values = method_search.predict(valid_set)
ct = np.array(pd.crosstab(pred_values, valid_set_values))
print(pred_values)
print(ct)
print(ct.trace()/ct.sum().sum())

[1 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 1
 0 0 1 0 1 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 

In [ ]:
#.696 is better than I had

In [43]:
#KNN pipe
from sklearn.neighbors import KNeighborsClassifier
full_pipe_KNN = Pipeline([
    ('imputer', SimpleImputer()), #there will, ultimately, be three options here.
        #mean, median, and node
    ('scaler', StandardScaler()), #this is statistical normalization (z - mu)/s.d.
    ('regressor', KNeighborsClassifier())  #need to specify number of neighbors. Do this in the grid search
])

param_grid = {
        'imputer__strategy': ["most_frequent","mean", "median"],
        'regressor__n_neighbors': [3,4,5],
        'regressor__algorithm': ['ball_tree', 'kd_tree', 'brute'],
        'regressor__weights':['uniform','distance'],
        'regressor__leaf_size':[1,2,3,4,5],
        'regressor__p':[1,2],
        }

method_search = GridSearchCV(
    full_pipe_KNN
    , param_grid
    , cv = 5
    , scoring = 'neg_mean_squared_error'
    , verbose = 4
    , n_jobs = 6)
method_search.fit(train_set, train_set_values)
print(method_search.best_params_)

Fitting 5 folds for each of 540 candidates, totalling 2700 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  14 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 160 tasks      | elapsed:    1.6s
[Parallel(n_jobs=6)]: Done 584 tasks      | elapsed:    6.4s
[Parallel(n_jobs=6)]: Done 1268 tasks      | elapsed:   13.5s
[Parallel(n_jobs=6)]: Done 2152 tasks      | elapsed:   23.1s


{'imputer__strategy': 'median', 'regressor__algorithm': 'ball_tree', 'regressor__leaf_size': 1, 'regressor__n_neighbors': 4, 'regressor__p': 2, 'regressor__weights': 'uniform'}


[Parallel(n_jobs=6)]: Done 2700 out of 2700 | elapsed:   29.3s finished


In [44]:
import numpy as np
pred_values = method_search.predict(valid_set)
ct = np.array(pd.crosstab(pred_values, valid_set_values))
print(pred_values)
print(ct)
print(ct.trace()/ct.sum().sum())

[0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1
 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 1 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0
 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
#.662 is a bit better, still not very good

In [77]:
#RF pipe
from sklearn.ensemble import RandomForestClassifier
full_pipe_RF = Pipeline([
    ('imputer', SimpleImputer()), #there will, ultimately, be three options here.
        #mean, median, and node
    ('scaler', StandardScaler()), #this is statistical normalization (z - mu)/s.d.
    ('regressor', RandomForestClassifier())  
])
#max depth

param_grid = {
        'imputer__strategy': ["most_frequent","mean", "median"],
        'regressor__n_estimators': [800,1000,1200],
        'regressor__min_samples_leaf':[1,2],
        'regressor__criterion': ['gini', 'entropy']
        }


method_search = GridSearchCV(
    full_pipe_RF
    , param_grid
    , cv = 5
    , scoring = 'neg_mean_squared_error'
    , verbose = 4
    , n_jobs = 6)
method_search.fit(train_set, train_set_values)
print(method_search.best_params_)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:   21.1s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 180 out of 180 | elapsed:  4.8min finished


{'imputer__strategy': 'median', 'regressor__criterion': 'gini', 'regressor__min_samples_leaf': 1, 'regressor__n_estimators': 800}


In [78]:
import numpy as np
pred_values = method_search.predict(valid_set)
ct = np.array(pd.crosstab(pred_values, valid_set_values))
print(pred_values)
print(ct)
print(ct.trace()/ct.sum().sum())

[0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0 1
 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0
 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [ ]:
#.688 is little improvement

In [63]:
#XGBoost pipe
from xgboost import XGBClassifier
full_pipe_XGB = Pipeline([
    ('imputer', SimpleImputer()), #there will, ultimately, be three options here.
        #mean, median, and node
    ('scaler', StandardScaler()), #this is statistical normalization (z - mu)/s.d.
    ('regressor', XGBClassifier())  
])
#max depth

param_grid = {
        'imputer__strategy': ["most_frequent","mean", "median"],
        'regressor__learning_rate': [.07,.05,.03],
        'regressor__max_depth': [30,35,40],
        'regressor__n_estimators': [40,50,60]
        }


method_search = GridSearchCV(
    full_pipe_XGB
    , param_grid
    , cv = 5
    , scoring = 'neg_mean_squared_error'
    , verbose = 4
    , n_jobs = 6)
method_search.fit(train_set, train_set_values)
print(method_search.best_params_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:    2.9s
[Parallel(n_jobs=6)]: Done  86 tasks      | elapsed:   19.8s
[Parallel(n_jobs=6)]: Done 209 tasks      | elapsed:   49.8s
[Parallel(n_jobs=6)]: Done 380 tasks      | elapsed:  1.5min
[Parallel(n_jobs=6)]: Done 405 out of 405 | elapsed:  1.7min finished
c:\users\t3\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:15:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
{'imputer__strategy': 'mean', 'regressor__learning_rate': 0.05, 'regressor__max_depth': 35, 'regressor__n_estimators': 50}


In [64]:
import numpy as np
pred_values = method_search.predict(valid_set)
ct = np.array(pd.crosstab(pred_values, valid_set_values))
print(pred_values)
print(ct)
print(ct.trace()/ct.sum().sum())

[0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 1 0 0 1
 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0
 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0
 0 1 0 1 1 0 0 0 0 0 1 1 1 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0
 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 0 0 1 0 0 0 0 0
 0 1 1 0 1 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0
 1 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 1 1 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 0 0 

In [76]:
#.685 on validation set with XGBoost? There has to be better.

In [ ]:
#I guess the support vector classifier is the best I'm going to get.
#Others on Kaggle have managed 85 percent though.
#May be that I'm overcomplicating things.